Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5903325 0.264796683 0.0041868032 0.0039207395 1000
 beta   0.9050437 0.042447637 0.0006711561 0.0007258681 1000
sigma   5.1082203 0.193088628 0.0030529993 0.0027348151 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 154.0750000 154.41600000 154.5890000 154.7660000 155.10800000
 beta   0.8227291   0.87670075   0.9045995   0.9331225   0.99018528
sigma   4.7371913   4.97663000   5.1030650   5.2351450   5.50035925



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[155.161 0.919375 5.44518; 154.874 0.922572 5.20084; … ; 154.189 0.905129 5.46606; 154.115 0.896906 5.12391]

Union{Missing, Float64}[154.551 0.862086 5.27424; 154.403 0.896538 5.53258; … ; 155.029 0.955062 4.97676; 155.143 0.956007 5.02748]

Union{Missing, Float64}[154.702 0.854888 5.03997; 154.57 0.965362 5.101; … ; 154.606 0.910535 5.074; 154.607 0.906699 5.0043]

Union{Missing, Float64}[155.09 1.00069 5.2901; 154.887 0.887246 4.92152; … ; 154.979 0.894451 5.73724; 154.045 0.960099 4.62501]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5903325 0.264796683 0.0041868032 0.0039207395 1000
 beta   0.9050437 0.042447637 0.0006711561 0.0007258681 1000
sigma   5.1082203 0.193088628 0.0030529993 0.0027348151 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 154.0750000 154.41600000 154.5890000 154.7660000 155.10800000
 beta   0.8227291   0.87670075   0.9045995   0.9331225   0.99018528
sigma   4.7371913   4.97663000   5.1030650   5.2351450   5.50035925



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*